In [ ]:
!git clone https://github.com/aliannejadi/ClariQ

Cloning into 'ClariQ'...
remote: Enumerating objects: 220, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 220 (delta 19), reused 32 (delta 14), pack-reused 176
Receiving objects: 100% (220/220), 253.36 MiB | 23.81 MiB/s, done.
Resolving deltas: 100% (110/110), done.
Updating files: 100% (41/41), done.


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 22.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel, DebertaModel, DebertaForSequenceClassification, DebertaTokenizer
from torch import nn
from torch.optim import Adam
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from transformers import set_seed

In [ ]:
datapath = '/content/ClariQ/data/train.tsv'
df_train = pd.read_csv(datapath, sep='\t')
df_train=df_train.drop_duplicates('initial_request')
df_train.head()

,topic_id,initial_request,topic_desc,clarification_need,facet_id,facet_desc,question_id,question,answer
0,1,Tell me about Obama family tree.,Find information on President Barack Obama\'s ...,2,F0001,"Find the TIME magazine photo essay ""Barack Oba...",Q00384,are you interested in seeing barack obamas family,yes am interested in obamas family
39,102,What is Fickle Creek Farm,Find general information about Fickle Creek Fa...,2,F0014,Find general information about Fickle Creek Fa...,Q00059,are you going to purchase anything there,i dont know yet i just want general info about...
84,105,Tell me about sonoma county medical services.,What medical services are available in Sonoma ...,2,F0025,What medical services are available in Sonoma ...,Q00457,are you interested in the human services depar...,no i am looking for doctors or hospitals in so...
120,108,Tell me about of Ralph Owen Brester.,Find biographical information about Ralph Owen...,1,F0037,Find biographical information about Ralph Owen...,Q00297,are you interested in learning more about ralp...,yes and his biography
159,109,I'm looking for information about mayo clinic ...,What medical services are available at the May...,2,F0040,What medical services are available at the May...,Q00256,are you interested in jobs at mayo clinic jack...,no im interested in services provided at mayo ...


In [ ]:
datapath = '/content/ClariQ/data/dev.tsv'
df_dev = pd.read_csv(datapath, sep='\t')
df_dev=df_dev.drop_duplicates('initial_request')
df_dev.head()

,topic_id,initial_request,topic_desc,clarification_need,facet_id,facet_desc,question_id,question,answer
0,101,Find me information about the Ritz Carlton Lak...,Find information about the Ritz Carlton resort...,2,F0010,Find information about the Ritz Carlton resort...,Q00697,are you looking for a specific web site,yes for the ritz carlton resort at lake las vegas
60,106,I'm looking for universal animal cuts reviews,Find testimonials of Universal Animal Cuts nut...,3,F0028,Find testimonials of Universal Animal Cuts nut...,Q01481,did universal animal cuts work for you,i need testimonials on the universal animal cu...
102,107,tell me about cass county missouri,Find demographic information about Cass County...,3,F0031,Find demographic information about Cass County...,Q00086,are you interested in a list of homes for sale...,no i want demographic info for cass county mo
192,114,Tell about an adobe indian house?,How does one build an adobe house?,2,F0063,How does one build an adobe house?,Q00057,are you going to purchase any specific product...,maybe
231,123,What is von Willebrand Disease?,What is von Willebrand Disease?,3,F0100,What is von Willebrand Disease?,Q00284,are you interested in learning about treatment...,id like to know what it is first


In [ ]:
df_train.groupby(['clarification_need'])
df_dev.groupby(['clarification_need'])
set_seed(42)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [label-1 for label in df['clarification_need']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 768, truncation=True,
                                return_tensors="pt") for text in df['initial_request']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [ ]:
class DebertaClassifier(nn.Module):

    def __init__(self, dropout=0.3):

        super(DebertaClassifier, self).__init__()

        self.bert =  DebertaModel.from_pretrained("microsoft/deberta-base",problem_type="multi_label_classification")
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 4)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        pooled_output = self.bert(input_ids= input_id, attention_mask=mask)
        pooled_output=pooled_output.last_hidden_state
        pooled_output=pooled_output[:,-1]
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)
        
        return final_layer

In [ ]:
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=4, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=4)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:
            torch.cuda.empty_cache()
            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)
                    
                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
def eval_result(preds, labels):
    """ Calculate the accuracy, f1, precision, recall of our predictions vs labels
    """

    y_pred = np.argmax(preds, axis=1).flatten()
    y_true = labels.flatten()

    precision = precision_score(y_true, y_pred,average='macro')
    recall = recall_score(y_true, y_pred,average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')
    accuracy = np.sum(y_pred == y_true) / len(y_true) 

    return (precision, recall, f1, accuracy)

from sklearn.metrics import classification_report

def eval_model( model, epoch,lr,test_dataloader):
    
    if torch.cuda.is_available():    
        device = torch.device("cuda")
        model.cuda()
    else:
        device = torch.device("cpu")


    model.eval()

    test_results = []
    test_labels = []
    test_results_predicted_lavels = [] 
    for test_input, test_label in tqdm(test_dataloader):

      test_label = test_label.to(device)
      mask = test_input['attention_mask'].to(device)
      input_id = test_input['input_ids'].squeeze(1).to(device)
       
      with torch.no_grad():        
        result = model(input_id,mask)

      # print(result)

      logits = result
      logits = logits.detach().cpu().numpy()
      test_results.extend(logits.tolist())
      tmp = np.asarray(logits.tolist())
      test_results_predicted_lavels.extend(np.argmax(tmp, axis=1).flatten())
      label_ids = test_label.to('cpu').numpy()
      test_labels.extend(label_ids)

    print(classification_report( np.asarray(test_labels), np.asarray(test_results_predicted_lavels)))
    (precision, recall, f1, accuracy) = eval_result(np.asarray(test_results), np.asarray(test_labels))

    print('Test Precision: {:.4f}, Test Recall: {:.4f}, Test Macro F1: {:.4f}, Test Accuracy: {:.4f} ' .format(precision, recall, f1, accuracy))

    # df = pd.DataFrame(test_results, columns=["0", "1"])

    # df['y_True'] = test_labels
    # #print(df.head())
    # df.to_csv( f"Final_Test_Probanility_Distribution_{model}_{epoch}e_{lr}lr_eval.csv", index=False)

In [ ]:

EPOCHS = 19
model = DebertaClassifier()
LR = 1e-6
              
train(model, df_train, df_dev, LR, EPOCHS)

Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaModel: ['lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.bias']
- This IS expected if you are initializing DebertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 47/47 [00:43<00:00,  1.08it/s]


Epochs: 1 | Train Loss:  0.322                 | Train Accuracy:  0.396                 | Val Loss:  0.317                 | Val Accuracy:  0.440


100%|██████████| 47/47 [00:41<00:00,  1.14it/s]


Epochs: 2 | Train Loss:  0.313                 | Train Accuracy:  0.422                 | Val Loss:  0.339                 | Val Accuracy:  0.400


100%|██████████| 47/47 [00:41<00:00,  1.12it/s]


...

Epochs: 17 | Train Loss:  0.102                 | Train Accuracy:  0.904                 | Val Loss:  0.326                 | Val Accuracy:  0.480


100%|██████████| 47/47 [00:45<00:00,  1.04it/s]


Epochs: 18 | Train Loss:  0.097                 | Train Accuracy:  0.947                 | Val Loss:  0.360                 | Val Accuracy:  0.420


100%|██████████| 47/47 [00:45<00:00,  1.04it/s]


Epochs: 19 | Train Loss:  0.089                 | Train Accuracy:  0.941                 | Val Loss:  0.350                 | Val Accuracy:  0.480


In [ ]:
datapath = '/content/ClariQ/data/test_with_labels.tsv'
df_test = pd.read_csv(datapath, sep='\t')
df_test = df_test.drop_duplicates('initial_request')
df_train.groupby(['clarification_need'])

In [ ]:
test = Dataset(df_test)
test_dataloader = torch.utils.data.DataLoader(test, batch_size=4, shuffle=True)
eval_model(model, EPOCHS , LR ,test_dataloader)

100%|██████████| 16/16 [00:06<00:00,  2.59it/s]

              precision    recall  f1-score   support

           0       0.50      0.12      0.20         8
           1       0.50      0.44      0.47        32
           2       0.34      0.56      0.43        18
           3       0.00      0.00      0.00         4

    accuracy                           0.40        62
   macro avg       0.34      0.28      0.27        62
weighted avg       0.42      0.40      0.39        62

Test Precision: 0.3362, Test Recall: 0.2795, Test Macro F1: 0.2730, Test Accuracy: 0.4032 


In [ ]:
val = Dataset(df_dev)
val_dataloader = torch.utils.data.DataLoader(val, batch_size=4, shuffle=True)
eval_model(model, EPOCHS , LR ,val_dataloader)

100%|██████████| 13/13 [00:04<00:00,  2.70it/s]

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.55      0.57      0.56        21
           2       0.40      0.62      0.49        16
           3       0.67      0.22      0.33         9

    accuracy                           0.48        50
   macro avg       0.40      0.35      0.34        50
weighted avg       0.48      0.48      0.45        50

Test Precision: 0.4030, Test Recall: 0.3547, Test Macro F1: 0.3448, Test Accuracy: 0.4800 



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classifica